#CONVERSION DE PDF EN MARKDOWN

In [ ]:
import nest_asyncio
import os
from pathlib import Path
from llama_parse import LlamaParse
import docx
import pandas as pd

In [ ]:

nest_asyncio.apply()

# Configuration  LlamaParse
parser = LlamaParse(
    api_key="llx...",
    result_type="markdown",
    verbose=True,
    show_progress=True,
    strict_mode_image_extraction=False,
    strict_mode_image_ocr=False,
)


In [ ]:
# Dossiers d'entrée et de sortie
input_dir = "./processus_branchement"
output_dir = "./markdown_branchements"

# Création du dossier de sortie si inexistant
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Fonction pour extraire le texte d'un fichier Word
def extract_word_text(file_path):
    doc = docx.Document(file_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n\n"
    return text

# Fonction pour extraire le texte d'un fichier Excel
def extract_excel_text(file_path):
    text = ""
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        text += f"## {sheet_name}\n\n"
        text += df.to_markdown(index=False) + "\n\n"
    return text

In [ ]:
# Parcourir récursivement les fichiers dans le dossier d'entrée
for root, dirs, files in os.walk(input_dir):
    for file in files:
        # Chemin complet du fichier d'entrée
        file_path = os.path.join(root, file)
        
        # Déterminer le chemin de sortie relatif
        relative_path = os.path.relpath(root, input_dir)
        output_subdir = os.path.join(output_dir, relative_path)
        os.makedirs(output_subdir, exist_ok=True)
        
        # Nom du fichier Markdown
        file_name = Path(file).stem
        output_path = os.path.join(output_subdir, f"{file_name}.md")
        
        # Vérifier si le fichier Markdown existe déjà
        if os.path.exists(output_path):
            print(f"Le fichier {output_path} existe déjà, conversion ignorée.")
            continue
        
        try:
            # Traitement selon le type de fichier
            if file.lower().endswith('.pdf'):
                # Conversion PDF en Markdown
                documents = parser.load_data(file_path)
                markdown_content = ""
                for doc in documents:
                    markdown_content += doc.text + "\n\n"
                    markdown_content += "---\n\n"
                    
            elif file.lower().endswith(('.docx', '.doc')):
                # Conversion Word en Markdown
                markdown_content = extract_word_text(file_path)
                
            elif file.lower().endswith(('.xlsx', '.xls')):
                # Conversion Excel en Markdown
                markdown_content = extract_excel_text(file_path)
                
            else:
                print(f"Type de fichier non pris en charge : {file_path}")
                continue
            
            # Écriture du fichier Markdown
            with open(output_path, "w", encoding="utf-8") as md_file:
                md_file.write(markdown_content)
            
            print(f"Fichier Markdown généré avec succès : {output_path}")
            
        except Exception as e:
            print(f"Erreur lors de la conversion de {file_path} : {str(e)}")

print("Conversion terminée.")